In [45]:
import os
import json
import pandas as pandas
import traceback

In [46]:
# from langchain_openai import ChatOpenAI

In [47]:
KEY=os.getenv("HUGGINGFACE_API_KEY")

In [48]:
# from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
# from dotenv import load_dotenv
# import os

# load_dotenv()

# llm = HuggingFaceEndpoint(
#     repo_id="mistralai/Mistral-7B-Instruct-v0.2",
#     huggingfacehub_api_token=KEY,
#     temperature=0.3,
#     max_new_tokens=256
# )

# chat = ChatHuggingFace(llm=llm)

# response = chat.invoke("Generate 3 MCQs about Python lists with answers")
# print(response.content)


In [49]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence, RunnableLambda
from langchain_core.callbacks import BaseCallbackHandler
import time
import PyPDF2

In [50]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [51]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [52]:
class TrackingCallback(BaseCallbackHandler):
    def on_llm_start(self, serialized, prompts, **kwargs):
        self.start_time = time.time()
        print("\n🔵 LLM STARTED")
        print("Prompt preview:\n", prompts[0][:500])

    def on_llm_end(self, response, **kwargs):
        print("\n🟢 LLM FINISHED")
        print(f"⏱ Time taken: {time.time() - self.start_time:.2f} seconds")

        try:
            print("\n📄 LLM OUTPUT:\n", response.generations[0][0].text)
        except Exception:
            pass

In [53]:
llm = ChatHuggingFace(
    llm = HuggingFaceEndpoint(
        repo_id="mistralai/Mistral-7B-Instruct-v0.2",
        huggingfacehub_api_token=KEY,
        temperature=0.3,
        max_new_tokens=600
    ),
    callbacks=[TrackingCallback()]
)

In [54]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [55]:
quiz_chain=(quiz_generation_prompt | llm)

In [56]:
quiz_evaluation_prompt=PromptTemplate(
    input_variables=["subject", "quiz"],
    template=TEMPLATE
)

In [57]:
review_chain=(
    RunnableLambda(
        lambda x: {
            "quiz": x,
            "subject": SUBJECT
        }
    )
    | quiz_evaluation_prompt 
    | llm)

In [58]:
generate_evaluate_chain = RunnableSequence(
    quiz_chain,
    RunnableLambda(lambda x: {"quiz": x, "subject": SUBJECT}),
    review_chain
)

In [59]:
# generate_evaluate_chain = (
#     quiz_generation_prompt
#     | llm
#     | RunnableLambda(
#         lambda x, inputs={
#             "text": TEXT,
#             "number": NUMBER,
#             "tone": TONE,
#             "response_json": json.dumps(RESPONSE_JSON),
#             "subject": SUBJECT
#         }: {
#             **inputs,           # carry forward all required inputs
#             "quiz": x.content    # add quiz output
#         }
#     )
#     | quiz_evaluation_prompt
#     | llm
# )


In [60]:
file_path=r"C:\Users\Admin\mcqgen\data.txt"

In [61]:
file_path

'C:\\Users\\Admin\\mcqgen\\data.txt'

In [62]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [63]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [64]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [65]:
NUMBER=5
SUBJECT="biology"
TONE="simple"

In [66]:
from huggingface_hub.utils import BadRequestError

for attempt in range(5):
    try:
        response = generate_evaluate_chain.invoke(
            {
                "text": TEXT,
                "number": NUMBER,
                "subject": SUBJECT,
                "tone": TONE,
                "response_json": json.dumps(RESPONSE_JSON)
            }
        )
        break
    except BadRequestError as e:
        print("Model not ready, retrying in 10s...")
        time.sleep(10)


🔵 LLM STARTED
Prompt preview:
 Human: 
Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows orga
Model not ready, retrying in 10s...

🔵 LLM STARTED
Prompt preview:
 Human: 
Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains th